In [ ]:
# import sys
# sys.path.append( '/anaconda/envs/py37_pytorch/lib/python3.7')
# sys.path.append('/anaconda/envs/py37_pytorch/lib/python3.7/site-packages')

In [1]:
%load_ext autoreload
%autoreload 2
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

import yaml
import glob
import torch
import numpy as np
from torch import nn
from torchvision.models import ResNet
from torch.utils.data import DataLoader,Dataset

import logging
from importlib import reload  # Not needed in Python 2
reload(logging)

torch.backends.cudnn.enabled = False
torch.backends.cudnn.deterministic = True
device = torch.device("cuda"  if torch.cuda.is_available() else "cpu")

logging.basicConfig(level=logging.INFO,#控制台打印的日志级别
                    filename='logging.txt',
                    filemode='a',##模式，有w和a，w就是写模式，每次都会重新写日志，覆盖之前的日志
                    #a是追加模式，默认如果不写的话，就是追加模式
                    format=
                    '%(asctime)s : %(message)s',
                    )



from dataset.ASdataset import AS_Data
from dataset.ASdataset_obs_train_input import AS_Data_obs


In [2]:
with open('config/cfg.yaml','r') as f:
    cfg = yaml.load(f)

cfg = {**cfg['step1'],**cfg['share_cfg']}
T = cfg['T']
pollution = cfg['pollution']
batch_size = cfg['batch_size']

print('train data is loading ')
Data = AS_Data(cfg['data_path'],left = cfg['train']['left'],right = cfg['train']['right'],window = T,pollution = pollution)
trainloader = DataLoader(Data,batch_size=batch_size,shuffle=True)
print(len(Data))

print('test data is loading ')
test_Data = AS_Data(cfg['data_path'],left = cfg['test']['left'],right = cfg['test']['right'],window = T,pollution = pollution)
testloader = DataLoader(test_Data,batch_size=batch_size,shuffle=True)
print(len(test_Data))

<ipython-input-2-fa3df4bc90e2>:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  cfg = yaml.load(f)


train data is loading 
/AS_data/Conc_npy/TOTAL_2015_01_NO2_SO2_O3_PM25_PM10_CO__744_6_182_232.npy   is loading
/AS_data/Conc_npy/TOTAL_2015_04_NO2_SO2_O3_PM25_PM10_CO__720_6_182_232.npy   is loading
/AS_data/Conc_npy/TOTAL_2015_07_NO2_SO2_O3_PM25_PM10_CO__744_6_182_232.npy   is loading
/AS_data/Conc_npy/TOTAL_2015_10_NO2_SO2_O3_PM25_PM10_CO__744_6_182_232.npy   is loading
(558, 4, 182, 232)
/AS_data/zeyuan_folder/concat_data/rest_EM_2015_01_744_07_182_232.npy   is loading
/AS_data/zeyuan_folder/concat_data/rest_EM_2015_04_720_07_182_232.npy   is loading
/AS_data/zeyuan_folder/concat_data/rest_EM_2015_07_744_07_182_232.npy   is loading
/AS_data/zeyuan_folder/concat_data/rest_EM_2015_10_744_07_182_232.npy   is loading
(558, 7, 182, 232)
/AS_data/METCRO2D_npy/METCRO2D_2015_01_744_34_182_232.npy   is loading
/AS_data/METCRO2D_npy/METCRO2D_2015_04_720_34_182_232.npy   is loading
/AS_data/METCRO2D_npy/METCRO2D_2015_07_744_34_182_232.npy   is loading
/AS_data/METCRO2D_npy/METCRO2D_2015_10_744

In [4]:
from model.res_model_LSTM import res8
from model.unet_model_LSTM import UNet

# test_model = res8(cfg['meteorological_dim']+cfg['emission_dim'],cfg['grid_dim'],T=T,pre_dim = len(pollution)) #+5*16
name = cfg['name']
test_model = UNet(cfg['meteorological_dim']+cfg['emission_dim'],cfg['grid_dim'],T=T,bilinear=False,pre_dim = len(pollution)) #+80
# name = cfg['name']
test_model.load_state_dict(torch.load('model_save/model_F_35_epoch.t'))

test_model.to(device)
criterion = torch.nn.L1Loss()
optimizer = torch.optim.Adam(test_model.parameters(),lr=1e-3)


In [9]:
def score(model,loader,criterion= nn.L1Loss(),percent = False):
    model.eval()
    ls = []
    for idx,i in enumerate(loader):
        with torch.no_grad():
            indexes,input,grid,yt_1,label,next_label, next_metro = [j.to(device) for j in i]
#             input,grid,yt_1,label,next_label, next_metro = input.to(device),grid.to(device),yt_1.to(device),label.to(device),next_label.to(device),next_metro.to(device)
            y_pred = model(input,grid,yt_1)
            
#             if NORM_METHOD == 1:
#                 y_pred = y_pred * (dic['max'] - dic['min']) + dic['min']
#                 label = label * (dic['max'] - dic['min']) + dic['min']
#             elif NORM_METHOD == 2:
#                 y_pred = (y_pred + dic['min']) * dic['std'] + dic['mean']
#                 label = label * (dic['max'] - dic['min']) + dic['min']
            
            cur_loss = []
            for j in range(label.shape[1]):
                if percent:
                    for esp in [0.1,1,4,8,12,16]:
                        loss = torch.mean(torch.abs(y_pred[:,j]-label[:,j])/(label[:,j]+esp))
                        cur_loss.append(loss.cpu().data)
                else:
                    loss = criterion(y_pred[:,j],label[:,j])
                    cur_loss.append(loss.cpu().data)

            ls.append(cur_loss)
            
    return np.mean(np.array(ls),axis = 0)

In [10]:
score(test_model,testloader,criterion = nn.L1Loss()) 

array([2.1765795 , 2.6870308 , 0.36613306, 0.27721682], dtype=float32)

In [158]:
string = '''
Apparent consumption of coal, oil and natural gas in China in 2013
was 3.84 Gt, 401.16 Mt and 131.30 Gm3
, respectively. Between 1997
and 2012, we estimate that cumulative energy consumption was 10% greater than the national statistics and 4% lower than provincial statistics (Extended Data Fig. 2). In addition, our results indicate a higher
annual growth rate of energy consumption than national statistics
between 2000 and 2010 (9.9% yr21 instead of 8.8% yr21
); the high
growth rate is consistent with satellite observations of NOx
20,21,
although NOx to fuel emission factors change with time as well.
Given the large fraction of CO2 emissions from coal combustion
(80% between 2000 and 2013), estimates of total emissions are heavily
dependent on the emission factors used to assess coal emissions. Thus,
we re-evaluate each of the variables that determine these emission
factors. The mean total carbon content of raw coal samples from
4,243 state-owned Chinese coal mines (4,243 mines represent 36%
of Chinese coal production in 2011)22 (Fig. 1) is 58.45% (Fig. 2a),
and the production-weighted total carbon content is 53.34%
'''

In [159]:
string.replace('\n',' ')

' Apparent consumption of coal, oil and natural gas in China in 2013 was 3.84 Gt, 401.16 Mt and 131.30 Gm3 , respectively. Between 1997 and 2012, we estimate that cumulative energy consumption was 10% greater than the national statistics and 4% lower than provincial statistics (Extended Data Fig. 2). In addition, our results indicate a higher annual growth rate of energy consumption than national statistics between 2000 and 2010 (9.9% yr21 instead of 8.8% yr21 ); the high growth rate is consistent with satellite observations of NOx 20,21, although NOx to fuel emission factors change with time as well. Given the large fraction of CO2 emissions from coal combustion (80% between 2000 and 2013), estimates of total emissions are heavily dependent on the emission factors used to assess coal emissions. Thus, we re-evaluate each of the variables that determine these emission factors. The mean total carbon content of raw coal samples from 4,243 state-owned Chinese coal mines (4,243 mines repres

In [ ]:
best_score = 1000
early_stop = 15
early_cnt = 0
for epoch in range(5,40):
#     logging.info('-----------{}-----------'.format(epoch))
    print('-----------{}-----------'.format(epoch))
    ls = []
    
    test_model.train()
    for idx,i in enumerate(trainloader):
        indexes,input,grid,yt_1,label,next_label, next_metro = [j.to(device) for j in i]
        
        y_pred = test_model(input,grid,yt_1)
#         print('*'*20)
        assert y_pred.shape == label.shape
        
        optimizer.zero_grad()
        
        loss = criterion(y_pred,label)
        loss.backward()
        optimizer.step()
        ls.append(loss.cpu().data)
        if len(ls)%40==0:
            logging.info('epoch {} cur loss {}'.format(epoch,np.mean(ls)))
            print('epoch {} cur loss {}'.format(epoch,np.mean(ls)))
#         torch.save(test_model.cpu().state_dict(),'model_save/test.t')
    
#     logging.info('epoch {} cur loss {}'.format(epoch,np.mean(ls)))
    print('epoch {} cur loss {}'.format(epoch,np.mean(ls)))
    test_score_L1 = score(test_model,testloader,criterion = nn.L1Loss()) 
#     logging.info('-------------cur test loss L1:  {}'.format(','.join([str(s) for s in test_score_L1])))
    print('-------------cur test loss L1:  {}'.format(','.join([str(s) for s in test_score_L1])))
    
    if epoch%5 == 0:
        torch.save(test_model.cpu().state_dict(),'model_save/model_F_{}_epoch.t'.format(epoch))
        test_model.to(device)
        
    if np.sum(test_score_L1)<best_score:
        early_cnt = 0
        best_score = np.sum(test_score_L1)
        torch.save(test_model.cpu().state_dict(),name)
        test_model.to(device)
    else:
        early_cnt += 1
        if early_cnt>=early_stop:
            break

In [ ]:
test_model.load_state_dict(torch.load(name))

# test_model = res8(51+34+16,27,[3],T=48)

test_model = test_model.to(device)

In [15]:
1

1

In [20]:
string = '''Jan. & 10.60 & 6.50 & 10.76 & 5.56 & 19.30 & 9.20 & 13.95 & 6.28 \\ \hline
        Apr. & 13.90 & 4.70 & 12.16 & 6.61 & 13.30 & 7.20 & 12.85 & 7.56 \\ \hline
        Jul. & 16.50 & 6.90 & 6.93 & 6.05 & 29.20 & 7.90 & 8.60 & 6.67 \\ \hline
        Oct. & 14.90 & 11.20 & 9.70 & 5.64 & 19.00 & 9.60 & 11.80 & 5.88\\ \hline 
        Average & 13.97&7.32&9.88&5.96&20.20&8.47&11.8&6.59 \\ \hline \hline'''

In [22]:
string = '''Jan. 	10.76	5.56	10.6	6.5	13.95	6.28	19.3	9.2
        Apr. 	12.16	6.61	13.9	4.7	12.85	7.56	13.3	7.2
        Jul. 	6.93	6.05	16.5	6.9	8.6	6.67	29.2	7.9
        Oct. 	9.7	5.64	14.9	11.2	11.8	5.88	19	9.6
        Average 	9.88	5.96	13.97	7.32	11.8	6.59	20.2	8.47
'''

In [25]:
print(string.replace('\t',' & '))

Jan.  & 10.76 & 5.56 & 10.6 & 6.5 & 13.95 & 6.28 & 19.3 & 9.2
        Apr.  & 12.16 & 6.61 & 13.9 & 4.7 & 12.85 & 7.56 & 13.3 & 7.2
        Jul.  & 6.93 & 6.05 & 16.5 & 6.9 & 8.6 & 6.67 & 29.2 & 7.9
        Oct.  & 9.7 & 5.64 & 14.9 & 11.2 & 11.8 & 5.88 & 19 & 9.6
        Average  & 9.88 & 5.96 & 13.97 & 7.32 & 11.8 & 6.59 & 20.2 & 8.47



In [21]:
for i in string.split('\n'):
    print('\t'.join(i.split('&')))

Jan. 	 10.60 	 6.50 	 10.76 	 5.56 	 19.30 	 9.20 	 13.95 	 6.28 \ \hline
        Apr. 	 13.90 	 4.70 	 12.16 	 6.61 	 13.30 	 7.20 	 12.85 	 7.56 \ \hline
        Jul. 	 16.50 	 6.90 	 6.93 	 6.05 	 29.20 	 7.90 	 8.60 	 6.67 \ \hline
        Oct. 	 14.90 	 11.20 	 9.70 	 5.64 	 19.00 	 9.60 	 11.80 	 5.88\ \hline 
        Average 	 13.97	7.32	9.88	5.96	20.20	8.47	11.8	6.59 \ \hline \hline


In [ ]:
plt.rcParams['figure.figsize'] = (16,7.0)

test_model.eval()
air_idx = 0 #pm25

for idx,i in enumerate(testloader):
    with torch.no_grad():
        input,grid,yt_1,label = i
        input = torch.squeeze(input,1)
        input,grid,yt_1,label = input.to(device),grid.to(device),yt_1.to(device),label.to(device)
        y_pred = test_model(input,grid,yt_1)
        
        label = label.cpu().numpy()
        y_pred = y_pred.cpu().numpy()
        print(label.shape)
        print(y_pred.shape)
        print(label)
        for b_idx in range(len(label)):
            
            plt.subplot(1,3,1)
            image1 = label[b_idx,air_idx]
            max_label = np.max(image1)
            image_show(image1,'label',base=max_label)
            
            plt.subplot(1,3,2)
            image2 = y_pred[b_idx,air_idx]
            image_show(image2,'pred',base=max_label)
            
            plt.subplot(1,3,3)
            image3 = np.abs(image2-image1)
            image_show(image3,'diff',base=max_label)
            plt.show()
        if idx>5:break
